In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd 'drive/My Drive/AI projects/DogsvsCats'

/content/drive/My Drive/AI projects/DogsvsCats


In [ ]:
import pandas as pd
data = pd.read_csv('data.csv')
data.head()

,filename,label
0,cat.9089.jpg,cat
1,cat.9110.jpg,cat
2,cat.9079.jpg,cat
3,cat.9096.jpg,cat
4,cat.909.jpg,cat


In [ ]:
import os
def check_exist(filename):
  if filename not in os.listdir('train'):
    return filename
data['filename'].apply(lambda x: check_exist(x))

In [ ]:
image_files = os.listdir('train')


In [ ]:
cat_data = data[data['label'] == 'cat'][:8000]
dog_data = data[data['label'] == 'dog'][:8000]
data = pd.concat([cat_data,dog_data])
data.reset_index(drop= True, inplace= True)

In [ ]:
from sklearn.model_selection import train_test_split
train_data, val_data = train_test_split(data, test_size= 0.2, random_state= 2020)
train_data.reset_index(drop= True, inplace= True)
val_data.reset_index(drop= True, inplace= True)
len(train_data), len(val_data)

(12800, 3200)

In [ ]:
# Custom dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from PIL import Image

class DogCatDataset(nn.Module):
  def __init__(self, dataframe, directory, transforms= None):
    self.df = dataframe
    self.dir = directory
    self.transforms= transforms
    self.class_dict = {'cat': 0 , 'dog': 1}
    self.df['class'] = self.df['label'].apply(lambda x: self.class_dict[x])
    self.IMAGE_SIZE = 128

  def __len__(self):
    return len(self.df)

  def __getitem__(self, index):
    image_file = self.df.loc[index, 'filename']
    image_path = os.path.join(self.dir, image_file)
    image = Image.open(image_path).convert('RGB').resize((self.IMAGE_SIZE, self.IMAGE_SIZE))

    label = torch.tensor(float(self.df.loc[index, 'class']))
    
    if self.transforms:
      image = self.transforms(image)

    return image, label

In [ ]:
# Transforms
from torchvision import transforms
IMAGE_MEAN = [0.5, 0.5, 0.5]
IMAGE_STD = [0.5, 0.5, 0.5]

train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(IMAGE_MEAN, IMAGE_STD)
])

val_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize(IMAGE_MEAN, IMAGE_STD)
])

In [ ]:
from torch.utils.data import DataLoader

TRAIN_DIR = 'train'
BATCH_SIZE = 32

train_ds = DogCatDataset(train_data, TRAIN_DIR, transforms= train_transforms)
val_ds = DogCatDataset(val_data, TRAIN_DIR, transforms= val_transforms)

train_dl = DataLoader(train_ds, batch_size= BATCH_SIZE, shuffle= True)
val_dl = DataLoader(val_ds, batch_size= BATCH_SIZE, shuffle= False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
class CNN(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3, 32, 3)
    # self.bn1 = nn.BatchNorm2d(126*126*32)
    self.conv2 = nn.Conv2d(32, 64, 3)
    # self.bn2 = nn.BatchNorm2d(61*61*64)
    self.conv3 = nn.Conv2d(64, 128, 3)
    # self.bn3 = nn.BatchNorm2d(28*28*128)
    self.flatten = nn.Flatten()
    self.linear1 = nn.Linear(14*14*128, 512)
    # self.bn4 = nn.BatchNorm1d(512)
    self.linear2 = nn.Linear(512, 1)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    # x = self.bn1(x)
    x = F.max_pool2d(x, (2,2))
    x = F.dropout(x, 0.25)

    x = self.conv2(x)
    x = F.relu(x)
    # x = self.bn2(x)
    x = F.max_pool2d(x, (2,2))
    x = F.dropout(x, 0.25)

    x = self.conv3(x)
    x = F.relu(x)
    # x = self.bn3(x)
    x = F.max_pool2d(x, (2,2))
    x = F.dropout(x, 0.25)

    x = self.flatten(x)
    x = self.linear1(x)
    x = F.relu(x)
    # x = self.bn4(x)
    x = F.dropout(x, 0.5)
    x = self.linear2(x)
    x = F.sigmoid(x)
    return x

In [ ]:
# Show a batch
for image_batch, label_batch in train_dl:
  print(image_batch.shape)
  print(label_batch)
  break

OSError: ignored

In [ ]:
# Show image
import matplotlib.pyplot as plt
import numpy as np
from torchvision.utils import make_grid

images, labels = next(iter(train_dl))
def show_image_batch(images):
  plt.figure(figsize = (16,24))
  grid_imgs = make_grid(images)
  np_grid_imgs = grid_imgs.numpy().transpose(1, 2, 0)
  np_grid_imgs = np_grid_imgs*np.array(IMAGE_STD) + np.array(IMAGE_MEAN)
  np_grid_imgs = np.clip(np_grid_imgs, 0, 1)
  plt.imshow(np_grid_imgs)

show_image_batch(images)

In [ ]:
device =  torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
IMAGE_SIZE = 128
model = CNN()
model = model.to(device)

In [ ]:
LR = 1e-3
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr= LR)

In [ ]:
train_losses = []
val_losses = []
val_accuracy_list = []
total_steps = []
step = 0
running_loss = 0
print_every = 5
num_epochs = 2

for epoch in range(num_epochs):
  for inputs, labels in train_dl:
    inputs, labels = inputs.to(device), labels.to(device)

    # Zero gradient
    optimizer.zero_grad()

    # Forward
    outputs = model(inputs).long()

    # Loss function
    loss = criterion(outputs, labels)

    # Backward
    loss.backward()

    # Update parameters
    optimizer.step()

    running_loss += loss.item()

    if step % print_every == 0:
      val_loss = 0
      val_accuracy = 0
      model.eval()
      with torch.no_grad():
        for inputs, labels in val_dl:
          inputs, labels = inputs.to(device), labels.to(device)

          # Forward
          predictions = model(inputs)

          # Val loss
          batch_loss = criterion(predictions, labels)

          val_loss += batch_loss.item()

          # Calculate accuracy
          class_pred = torch.where(predictions < 0.5, torch.tensor(0.), torch.tensor(1.))
          val_accuracy += torch.sum(class_pred == labels).item()/len(val_data)

      train_losses.append(running_loss/print_every)
      val_losses.append(val_loss/len(val_dl))
      val_accuracy_list.append(val_accuracy)
      total_steps.append(step)
      print(f'Device {device}.. Epoch {epoch+1}/{num_epoch}.. Step {step}.. Train loss: {running_loss/print_every:.3f}.. val loss: {val_loss/len(val_dl):.3f}.. val acc: {val_accuracy:.3f}')
      running_loss = 0
      model.train()

